In [1]:
%reset -f
%matplotlib inline

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
dirToInclude=parentdir +'/features/'
sys.path.insert(0,dirToInclude)


import IeegConsts
from IeegConsts import *
from IeegFeatures import *

import pandas
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split
import matplotlib.pyplot as plt


%matplotlib inline

np.set_printoptions(precision=4, threshold=10000, linewidth=100, edgeitems=999, suppress=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 100)
pd.set_option('expand_frame_repr', False)
pd.set_option('precision', 6)
    
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train_dir=TRAIN_DATA_FOLDER_IN_ALL
test_dir=TEST_DATA_FOLDER_IN_ALL    

ieegFeatures= IeegFeatures(train_dir, True)
df_cols_train=ieegFeatures.ieegGenCols()
print len(df_cols_train)
F_NAME_TRAIN= TRAIN_FEAT_BASE + TRAIN_PREFIX_ALL +'-feat_TRAIN_df.csv'
X_df_train=pandas.read_csv(F_NAME_TRAIN, engine='python') 
X_df_train.drop('Unnamed: 0', axis=1, inplace=True)


def dropBadFiles(df):
    print df.shape    

    s1 = ['1_137_0.mat', '1_138_0.mat', '1_139_0.mat', '1_140_0.mat', '1_141_0.mat', '1_142_0.mat', '1_166_0.mat', '1_167_0.mat', '1_168_0.mat', '1_169_0.mat', '1_170_0.mat',
          '1_171_0.mat', '1_266_0.mat', '1_267_0.mat', '1_268_0.mat', '1_269_0.mat', '1_270_0.mat', '1_271_0.mat', '1_303_0.mat', '1_304_0.mat', '1_305_0.mat', '1_306_0.mat',
          '1_307_0.mat', '1_308_0.mat', '1_397_0.mat', '1_398_0.mat', '1_399_0.mat', '1_400_0.mat', '1_401_0.mat', '1_402_0.mat', '1_412_0.mat', '1_413_0.mat', '1_414_0.mat',
          '1_415_0.mat', '1_416_0.mat', '1_417_0.mat', '1_481_0.mat', '1_482_0.mat', '1_483_0.mat', '1_484_0.mat', '1_485_0.mat', '1_486_0.mat', '1_520_0.mat', '1_521_0.mat',
          '1_522_0.mat', '1_523_0.mat', '1_524_0.mat', '1_525_0.mat', '1_585_0.mat', '1_586_0.mat', '1_587_0.mat', '1_588_0.mat', '1_589_0.mat', '1_590_0.mat', '1_621_0.mat',
          '1_622_0.mat', '1_623_0.mat', '1_624_0.mat', '1_625_0.mat', '1_626_0.mat', '1_703_0.mat', '1_704_0.mat', '1_705_0.mat', '1_706_0.mat', '1_707_0.mat', '1_708_0.mat',
          '1_784_0.mat', '1_785_0.mat', '1_786_0.mat', '1_787_0.mat', '1_788_0.mat', '1_789_0.mat', '1_855_0.mat', '1_856_0.mat', '1_857_0.mat', '1_858_0.mat', '1_859_0.mat',
          '1_860_0.mat', '1_985_0.mat', '1_986_0.mat', '1_987_0.mat', '1_988_0.mat', '1_989_0.mat', '1_990_0.mat', '1_1015_0.mat', '1_1016_0.mat', '1_1017_0.mat', '1_1018_0.mat',
          '1_1019_0.mat', '1_1020_0.mat', '1_1099_0.mat', '1_1100_0.mat', '1_1101_0.mat', '1_1102_0.mat', '1_1103_0.mat', '1_1104_0.mat', '1_1129_0.mat', '1_1130_0.mat', '1_1131_0.mat',
          '1_1133_0.mat', '1_1134_0.mat']

    s2 = ['2_69_0.mat', '2_70_0.mat', '2_71_0.mat', '2_72_0.mat', '2_399_0.mat', '2_400_0.mat', '2_401_0.mat', '2_402_0.mat', '2_439_0.mat', '2_440_0.mat', '2_441_0.mat',
          '2_442_0.mat', '2_443_0.mat', '2_444_0.mat', '2_452_0.mat', '2_453_0.mat', '2_454_0.mat', '2_455_0.mat', '2_456_0.mat', '2_531_0.mat', '2_532_0.mat', '2_533_0.mat',
          '2_534_0.mat', '2_763_0.mat', '2_764_0.mat', '2_765_0.mat', '2_766_0.mat', '2_767_0.mat', '2_768_0.mat', '2_1427_0.mat', '2_1428_0.mat', '2_1429_0.mat', '2_1430_0.mat',
          '2_1431_0.mat', '2_1432_0.mat', '2_1603_0.mat', '2_1604_0.mat', '2_1605_0.mat', '2_1763_0.mat', '2_1764_0.mat', '2_1765_0.mat', '2_1766_0.mat', '2_1767_0.mat', '2_2119_0.mat',
          '2_2120_0.mat', '2_2121_0.mat', '2_2122_0.mat', '2_2123_0.mat', '2_2124_0.mat']

    s3 = ['3_799_0.mat', '3_901_0.mat', '3_902_0.mat', '3_903_0.mat', '3_904_0.mat', '3_905_0.mat', '3_906_0.mat',
          '3_1105_0.mat', '3_1106_0.mat', '3_1107_0.mat', '3_1108_0.mat']
    
    for item in s1:
        print 'droping:' + item
        df = df.drop(df[df.file == item].index)

    for item in s2:
        print 'droping:' + item
        df = df.drop(df[df.file == item].index)

    for item in s3:
        print 'droping:' + item
        df = df.drop(df[df.file == item].index)

    print 'Final shape:' + str(df.shape)
    return df


last_cols=list()
for i in range(1, 160 + 1):
    last_cols.append('psd_{}'.format(i))    
for i in range(1, 16 + 1):
    last_cols.append('var_{}'.format(i))    
for i in range(1, 16 + 1):
    last_cols.append('kurt_{}'.format(i))
for i in range(1, n_corr_coeff + 1):
    last_cols.append('corcoef_{}'.format(i))
    # for i in range(1, n + 1):
    #     cols.append('hurst_{}'.format(i))
for i in range(1,  n_plv+ 1):
    last_cols.append('plv_{}'.format(i))    

        
# X_df_train_SINGLE=X_df_train
# X_df_train_SINGLE = dropBadFiles(X_df_train)

# answers_1_SINGLE = list (X_df_train_SINGLE[singleResponseVariable].values)
# X_df_train_SINGLE = X_df_train_SINGLE.drop(singleResponseVariable, axis=1)
# X_df_train_SINGLE.drop('id', axis=1, inplace=True)
# X_df_train_SINGLE.drop('file', axis=1, inplace=True)
# X_df_train_SINGLE.drop('patient_id', axis=1, inplace=True)
# X_df_train_SINGLE.drop('file_size', axis=1, inplace=True)
# X_df_train_SINGLE.drop('sequence_id', axis=1, inplace=True)
# X_df_train_SINGLE.drop('segment', axis=1, inplace=True)

# X_df_train_SINGLE=X_df_train_SINGLE[last_cols]
# X_df_train_SINGLE=X_df_train_SINGLE.apply(lambda x: pandas.to_numeric(x, errors='ignore'))


# # Build pasty expression -- feed best features automatically 
# s='result ~ '

# last_cols=['corcoef_101', 'corcoef_106', 'corcoef_114', 'corcoef_24', 'corcoef_30', 'corcoef_42', 'corcoef_43', 'corcoef_54', 'corcoef_57', 'corcoef_67', 'corcoef_78', 'psd_103', 'psd_105', 'psd_132', 'psd_142', 'psd_145', 'psd_151', 'psd_152', 'psd_35', 'psd_36', 'psd_45', 'psd_81', 'psd_97', 'psd_98', 'var_1', 'var_11', 'var_16', 'var_3', 'var_4', 'var_9']
# for item in last_cols:
#     s += item + ' + '
# s=s[:-2]
# print s

Starting:ieegFeatures:2016-10-31 18:41:36.239124
Cols:1239
1239


In [ ]:
# Pymc expects an R like Df, so the response variable must be included
ieegFeatures= IeegFeatures(train_dir, True)
df_cols_train=ieegFeatures.ieegGenCols()
print len(df_cols_train)
F_NAME_TRAIN= TRAIN_FEAT_BASE + TRAIN_PREFIX_ALL +'-feat_TRAIN_df.csv'
X_df_train=pandas.read_csv(F_NAME_TRAIN, engine='python') 
X_df_train.drop('Unnamed: 0', axis=1, inplace=True)
X_df_train_SINGLE=X_df_train
_df_train_SINGLE = dropBadFiles(X_df_train)

X_df_train_SINGLE.drop('id', axis=1, inplace=True)
X_df_train_SINGLE.drop('file', axis=1, inplace=True)
X_df_train_SINGLE.drop('patient_id', axis=1, inplace=True)
X_df_train_SINGLE.drop('file_size', axis=1, inplace=True)
X_df_train_SINGLE.drop('sequence_id', axis=1, inplace=True)
X_df_train_SINGLE.drop('segment', axis=1, inplace=True)
# X_df_train_SINGLE=X_df_train_SINGLE[last_cols]
X_df_train_SINGLE=X_df_train_SINGLE.apply(lambda x: pandas.to_numeric(x, errors='ignore'))

from scipy.special import expit

def fastPredict(new_observation, theta, intercept):    
    v =  intercept + np.einsum('j,j->',new_observation, theta)    
    return expit(v)

def plot_traces(traces, retain=1000):    
    ax = pm.traceplot(traces[-retain:], figsize=(12,len(traces.varnames)*1.5),
        lines={k: v['mean'] for k, v in pm.df_summary(traces[-retain:]).iterrows()})

    for i, mn in enumerate(pm.df_summary(traces[-retain:])['mean']):
        ax[i,0].annotate('{:.2f}'.format(mn), xy=(mn,0), xycoords='data'
                    ,xytext=(5,10), textcoords='offset points', rotation=90
                    ,va='bottom', fontsize='large', color='#AA0022')
        
# import pymc3 as pm
import pymc3
from pymc3.glm import glm
from pymc3.glm.families import Binomial

niter=200000
i=0
with pymc3.Model() as logistic_model:
    pymc3.glm.glm(s, X_df_train_SINGLE, family=pymc3.glm.families.Binomial())
    print 'Iteration:' + str(i)
    i=i+1
    step = pymc3.NUTS()
    trace_logistic_model = pymc3.sample(niter, step=step,progressbar=True)    
    
# predict
df_trace_logistic_model = pm.trace_to_dataframe(trace_logistic_model[niter//2:])
w_theta = df_trace_logistic_model[last_cols].mean(0)
df_trace_logistic_model.to_csv("df_trace_logistic_model.csv")
w_theta.to_csv("w_theta.csv")
w_intercept=df_trace_logistic_model['Intercept'].mean(0)

print w_theta
print w_intercept 
plot_traces(trace_logistic_model, retain=100)

Starting:ieegFeatures:2016-10-15 18:26:40.611235
Cols:559
559
Iteration:0


## Submition

In [4]:
from scipy.special import expit

def fastPredict(new_observation, theta, intercept):    
    v =  intercept + np.einsum('j,j->',new_observation, theta)    
    return expit(v)


test_dir=TEST_DATA_FOLDER_IN_ALL
ieegFeatures= IeegFeatures(test_dir, False)
df_cols_test=ieegFeatures.ieegGenCols()
print len(df_cols_test)
F_NAME_TEST= TEST_FEAT_BASE + TEST_PREFIX_ALL +'-feat_TEST_df.csv'
X_df_TEST=pandas.read_csv(F_NAME_TEST, engine='python') 
X_df_TEST.drop('Unnamed: 0', axis=1, inplace=True)
# X_df_TEST.drop('id', axis=1, inplace=True)
X_df_TEST.drop('file', axis=1, inplace=True)
X_df_TEST.drop('patient_id', axis=1, inplace=True)
# X_df_TEST.drop('file_size', axis=1, inplace=True)
# X_df_TEST.drop('sequence_id', axis=1, inplace=True)
X_df_TEST.head(3)

#------------------------------------------------------------------------------#
now = datetime.now()
import dis
sub_file = 'submission' + '_mcmc_'  + str(len(glm_factor)) + '_' + str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv'

r= pandas.DataFrame.from_csv('sample_submission.csv')
print('Writing submission: ', sub_file)
f = open(sub_file, 'w') # append mode
f.write('File,Class\n')
total = 0

for index, row in r.iterrows():            
    id_str= index     
    arr = id_str.split("_")
    patient = int(arr[0])        
    new_id= ieegFeatures.getIdFromFileName(id_str) 
#     print str(new_id)
    
    X_df_single_row=X_df_TEST.loc[X_df_TEST['id'] == new_id]
    X_df_single_row.drop('id', axis=1, inplace=True)
    X_df_single_row= X_df_single_row[glm_factor]        
#     X_df_single_row.drop('file', axis=1, inplace=True)
#     X_df_single_row.drop('patient_id', axis=1, inplace=True)                    
    X_df_single_row = np.asarray(X_df_single_row)        
    c_pred= fastPredict(tuple (X_df_single_row)[0]),w_intercept, w_theta))
    str1 = id_str + ',' + str(predict (tuple (X_df_single_row)[0])) + '\n'  
#     print str1
    
    f.write(str1)
    
f.close()

print('Done writing submission: ', sub_file)

Starting:ieegFeatures:2016-09-22 16:51:05.199385
Cols:499
499
('Writing submission: ', 'submissionmcmc_all_5022016-09-22-16-51.csv')
('Done writing submission: ', 'submissionmcmc_all_5022016-09-22-16-51.csv')
